In [ ]:
from io import BytesIO
import json
import PIL.Image
import numpy as np
import requests

from velour.client import Client, ClientException
from velour.data_types import GroundTruthInstanceSegmentation, GroundTruthSemanticSegmentation, Image, Label
from velour.viz import combined_segmentation_mask
from velour.integrations.coco import upload_coco_panoptic
from velour.integrations.yolo import parse_yolo_object_detection, parse_yolo_image_segmentation
from velour.metrics import Task

from ultralytics import YOLO

In [ ]:
from chariot.datasets import Dataset

In [ ]:
client = Client("http://localhost:8000")

# Dataset ingestion

We assume the COCO panoptic evaluations from
http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip have been downloaded and unzipped (including unzipping the file `panoptic_val2017.zip`) to `./coco`

In [ ]:
with open("./coco/annotations/panoptic_val2017.json") as f:
    annotations = json.load(f)
    
annotations["annotations"] = annotations["annotations"][:10]

In [ ]:
# helper method to download underlying images from COCO
image_id_to_coco_url = {str(img_dict["id"]): img_dict["coco_url"] for img_dict in annotations["images"]}
def download_image(img: Image) -> PIL.Image:
    url = image_id_to_coco_url[img.uid]
    img_data = BytesIO(requests.get(url).content)
    return PIL.Image.open(img_data)

In [ ]:
# creates a dataset object in velour
try:
    dataset = client.create_image_dataset("coco2017-panoptic")
    upload_coco_panoptic(
        dataset,
        annotations=annotations,
        masks_path="./coco/annotations/panoptic_val2017/"
    )
except ClientException:
    dataset = client.get_dataset("coco2017-panoptic")

In [ ]:
dataset.get_images()

In [ ]:
download_image(dataset.get_images()[0])

In [ ]:
instance_segs = dataset.get_groundtruth_instance_segmentations("139")
semantic_segs = dataset.get_groundtruth_semantic_segmentations("139")

In [ ]:
instance_mask, instance_legend = combined_segmentation_mask(instance_segs, label_key="name")

In [ ]:
instance_mask

In [ ]:
for k, v in instance_legend.items():
    print(k)
    display(v)

In [ ]:
semantic_mask, semantic_legend = combined_segmentation_mask(semantic_segs, label_key="name")

In [ ]:
semantic_mask

In [ ]:
for k, v in semantic_legend.items():
    print(k)
    display(v)

# Model inference

In [ ]:
from chariot.client import connect
from chariot.models import Model
from tqdm.auto import tqdm

from velour.integrations.chariot import create_model_from_chariot, parse_chariot_object_detections

In [ ]:
connect("https://production.chariot.striveworks.us/")

In [ ]:
dataset = client.get_dataset("coco2017-panoptic")

In [ ]:
model_name = "fasterrcnnresnet-50fpn"
chariot_model = Model(name=model_name, project_name="Global")

try:
    velour_chariot_model = create_model_from_chariot(client, chariot_model)
except ClientException:
    velour_chariot_model = client.get_model('chariot-'+model_name+'-v'+str(chariot_model.version))


try:
    for image_metadata in tqdm(dataset.get_images()):
        image = download_image(image_metadata)    
        
        # Chariot Inference
        detections = chariot_model.detect(image)
        velour_dets = parse_chariot_object_detections(detections, image_metadata, label_key="name") 

        velour_chariot_model.add_predictions(dataset=dataset, predictions=velour_dets, show_progress_bar=False)
except ClientException:
    print("Velour Chariot model already finalized.")
    

In [ ]:
model_name = "yolov8n-seg"
yolo_model = YOLO(f"{model_name}.pt")

try:
    velour_yolo_model = client.create_image_model(model_name + "")
except ClientException:
    velour_yolo_model = client.get_model(model_name + '')

try:
    for image_metadata in tqdm(dataset.get_images()):
        image = download_image(image_metadata)     
        
        # YOLO Inference
        results = yolo_model(image, verbose=False)
        
        if model_name == 'yolov8n-seg':
            velour_dets = parse_yolo_object_detection(results[0], uid=image_metadata.uid, label_key='name')
        # elif model_name == 'yolov8n-seg':
        #     velour_dets = parse_yolo_image_segmentation(results[0], uid=image_metadata.uid, label_key='name')
        else:
            raise NameError

        velour_yolo_model.add_predictions(dataset=dataset, predictions=velour_dets, show_progress_bar=False)

except ClientException:
    print("Velour YOLO model already finalized.")

In [ ]:
print(velour_dets[0].scored_labels)

# Evaluation

for provenance, the dataset and inferences on the dataset must be finalized before evaluation can be done

In [ ]:
dataset.finalize()
velour_chariot_model.finalize_inferences(dataset)
velour_yolo_model.finalize_inferences(dataset)

In [ ]:
import time

def evaluate_ap(dataset, model, max_area=None):
    eval_job = model.evaluate_ap(
        dataset=dataset,
        model_pred_task_type=Task.BBOX_OBJECT_DETECTION,
        dataset_gt_task_type=Task.INSTANCE_SEGMENTATION,
        label_key="name",
        max_area=max_area,
    )

    while eval_job.status() == 'Processing':
        time.sleep(1)
    
    return eval_job

In [ ]:
job1 = evaluate_ap(dataset, velour_chariot_model)
job2 = evaluate_ap(dataset, velour_chariot_model, max_area=30*300)

In [ ]:
job3 = evaluate_ap(dataset, velour_yolo_model)
job4 = evaluate_ap(dataset, velour_yolo_model, max_area=30*300)

In [ ]:
print(job1.metrics()[:2])

In [ ]:
print(job2.metrics()[:2])

In [ ]:
print(job3.metrics()[:2])

In [ ]:
print(job4.metrics()[:2])

# Metadata

In [26]:
from velour.client import Client
from velour.data_types import Label
client = Client("http://localhost:8000")

velour_coco = client.get_dataset("coco2017-panoptic")

model_name = "yolov8n-seg"
velour_yolo = client.get_model(model_name + '')

model_name = "fasterrcnnresnet-50fpn"
velour_chariot = client.get_model('chariot-'+model_name+'-v'+str('1'))

Succesfully connected to http://localhost:8000/.


In [27]:
print(velour_coco.name)
print(velour_coco.description)
print(velour_coco.href)
print(velour_coco.number_of_classifications)
print(velour_coco.number_of_detections)
print(velour_coco.number_of_segmentations)
print(velour_coco.type)
print(velour_coco.associated_models)

coco2017-panoptic
None
None
0
0
351
SEGMENTATION
['chariot-fasterrcnnresnet-50fpn-v1', 'yolov8n-seg']


In [28]:
print(velour_yolo.name)
print(velour_yolo.description)
print(velour_yolo.href)
print(velour_yolo.number_of_classifications)
print(velour_yolo.number_of_detections)
print(velour_yolo.number_of_segmentations)
print(velour_yolo.type)
print(velour_yolo.associated_datasets)

yolov8n-seg
None
None
0
49
0
DETECTION
['coco2017-panoptic']


In [29]:
print(velour_chariot.name)
print(velour_chariot.description)
print(velour_chariot.href)
print(velour_chariot.number_of_classifications)
print(velour_chariot.number_of_detections)
print(velour_chariot.number_of_segmentations)
print(velour_chariot.type)
print(velour_chariot.associated_datasets)

chariot-fasterrcnnresnet-50fpn-v1
FasterRCNNResnet50FPN detector pretrained on COCO. Transferred from torchvision.models. This model was uploaded via CICD
https://production.chariot.striveworks.us/projects/2LVnCYT9WAMXfIZjW2o4E1X3Mvz/models/2LhPalERvF0qb42znK7D35fOwjI
0
96
0
DETECTION
['coco2017-panoptic']


In [ ]:
distribution = velour_coco.get_label_distribution()

label_person = Label(key='name', value='person')
label = Label(key='supercategory', value='furniture')

count = distribution[label_person]

print(count)

In [ ]:
distribution = velour_yolo.get_label_distribution()

label = Label(key='name', value='tv')

info = distribution[label]

print(info['count'])
print(info['scores'])

In [ ]:
labels = velour_yolo.get_labels()

for label in labels:
    print(label)